In [1]:
import csv
import gzip

from s2ag_corpus.database_catalogue import local_connection

In [2]:
connection = local_connection()

In [3]:
CREATE_PAPERIDS = """
create table if not exists paperids
(
    sha        text    not null
        constraint paperids_pk
            unique,
    corpusid   text    not null,
    is_primary boolean not null
)"""

In [4]:
cursor = connection.cursor()

In [5]:
cursor.execute('drop table if exists paperids')
cursor.execute(CREATE_PAPERIDS)
connection.commit()

In [6]:
csv_file_path = "../../data/diffs/initial-diff-data-for-test-db.csv" 
with open(csv_file_path, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        cursor.execute(
            f"INSERT INTO paperids (sha, corpusid, is_primary) VALUES (%s, %s, %s)",
            row
        )
connection.commit()

In [7]:
connection.rollback()

In [8]:
import json
from s2ag_corpus.datasets import Dataset

In [9]:


def upsert(connection, dataset: Dataset, path_to_file):
    cursor = connection.cursor()
    with gzip.open(path_to_file, 'rt') as upsert_file:
        for (index, line) in enumerate(upsert_file):
            row = dataset.json_to_tuple(line)
            print(row)
            cursor.execute(dataset.upsert, row)
            if 0 == index % 10000:
                connection.commit()
        connection.commit()

In [10]:
from s2ag_corpus.datasets import DATASETS

In [11]:

upsert(connection, DATASETS['paper-ids'], '/home/romilly/git/active/s2ag-corpus/data/diffs/update_files-000.gz')

('update01', 1234567, 'true')
('insert01', 2345678, 'true')
('insert02', 35791, 'false')
('update02', 27463, 'false')


In [12]:
def delete_rows(connection, dataset, path_to_file):
    with gzip.open(path_to_file, 'rt') as upsert_file:
        for (index, line) in enumerate(upsert_file):
            key = json.loads(line)[dataset.primary_key]
            print(key)
            cursor.execute(dataset.delete_row, (key,))
            if 0 == index % 10000:
                connection.commit()
        connection.commit()
    

In [13]:
delete_rows(connection, DATASETS['paper-ids'], 
            '/home/romilly/git/active/s2ag-corpus/data/diffs/delete_files-000.gz')

delete01
delete02


In [14]:
def read_csv_file(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)
        data = [(sha, corpusid, is_primary in ['True', 'true']) for sha, corpusid, is_primary in data]
        data.sort(key=lambda x: x[0])
        return data

In [15]:
expected = read_csv_file('/home/romilly/git/active/s2ag-corpus/data/diffs/expected-table-contents.csv')

In [16]:
expected

[('insert01', '2345678', True),
 ('insert02', '35791', False),
 ('unchanged01', '24680', False),
 ('unchanged02', '86429', True),
 ('update01', '1234567', True),
 ('update02', '27463', False)]

In [17]:
def fetch_data(cursor):
    query = "SELECT * FROM paperids ORDER BY sha"
    cursor.execute(query)
    return cursor.fetchall()

In [18]:
actual = fetch_data(cursor)

In [19]:
actual

[('insert01', '2345678', True),
 ('insert02', '35791', False),
 ('unchanged01', '24680', False),
 ('unchanged02', '86429', True),
 ('update01', '1234567', True),
 ('update02', '27463', False)]

In [20]:
for e,a in zip(expected, actual):
    if e != a:
        print(f"{e} != {a}")